In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Downloading files

In [2]:
!curl -O "https://raw.githubusercontent.com/SaminRK/scPSS-reproducibility/refs/heads/master/data_downloaders/download_calcagno.sh"
!curl -O "https://raw.githubusercontent.com/SaminRK/scPSS-reproducibility/refs/heads/master/preprocessors/preprocess_calcagno.py"
!curl -O "https://raw.githubusercontent.com/SaminRK/scPSS-reproducibility/refs/heads/master/utils/mean_margin.py"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1445  100  1445    0     0   7655      0 --:--:-- --:--:-- --:--:--  7686
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5416  100  5416    0     0  26819      0 --:--:-- --:--:-- --:--:-- 26945
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   987  100   987    0     0   5923      0 --:--:-- --:--:-- --:--:--  5945


In [3]:
!chmod +x download_calcagno.sh

In [4]:
!./download_calcagno.sh data

==> Downloading Zenodo ZIP...
--2025-07-24 05:43:26--  https://zenodo.org/records/7055957/files/Nikatag/Single-Cell-Spatial-Transcriptomics-for-Border-zone-BZ_Cell_Mapping.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7972177 (7.6M) [application/octet-stream]
Saving to: ‘/content/data/Single-Cell-Spatial-Transcriptomics-for-Border-zone-BZ_Cell_Mapping_zenodo.zip’

/content/data/Singl 100%[===================>]   7.60M   877KB/s    in 9.4s    

2025-07-24 05:43:36 (824 KB/s) - ‘/content/data/Single-Cell-Spatial-Transcriptomics-for-Border-zone-BZ_Cell_Mapping_zenodo.zip’ saved [7972177/7972177]

==> Unzipping Zenodo ZIP...
Archive:  /content/data/Single-Cell-Spatial-Transcriptomics-for-Border-zone-BZ_Cell_Mapping_zenodo.zip
8f7e1d14be5fa874c92bff1d8cb282eb9aa613fd
   creating: /content/data/Nikatag-Single-Cel

## scPSS

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
!pip install git+https://github.com/SaminRK/scPSS.git

  Cloning https://github.com/SaminRK/scPSS.git to /tmp/pip-req-build-8og28vt3
  Running command git clone --filter=blob:none --quiet https://github.com/SaminRK/scPSS.git /tmp/pip-req-build-8og28vt3
  Resolved https://github.com/SaminRK/scPSS.git to commit c16c71713e6a618ed6196b83b1dc3daa30838e1a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of umap-learn to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 4.4 MB/s eta 0:00:00
  Created wheel for scPSS: fi

In [7]:
DATA_DIR = 'data'

In [8]:
## Modify to the directory where outputs should be stored
OUT_PATH = "/content/drive/MyDrive/Research - Sohel sir/Pathological Cells/Pathological Cell Identification/reproducibility/outputs"
metrics_csv_path = f"{OUT_PATH}/scpss-mouse-infarcted-heart-metrics.csv"
results_csv_path = f"{OUT_PATH}/scpss-mouse-infarcted-heart-results.csv"

In [9]:
from scpss import scPSS
import time
import numpy as np
from preprocess_calcagno import get_preprocessed_anndata


with open(metrics_csv_path, 'w') as f:
    print('PROBLEM_KEY', 'seed', 'accuracy', 'precision', 'recall', 'f1', 'auc', 'aupr', 'outlier_ratio_ref', 'outlier_ratio_que', 'time_taken',  sep=',', file=f, flush=True)

    for PROBLEM_KEY in ['ref_0_que_1hr', 'ref_0_que_4hr', 'ref_0_que_1', 'ref_0_que_3', 'ref_0_que_7',  'ref_0_que_1-7']:
        ad = get_preprocessed_anndata(PROBLEM_KEY, DATA_DIR)
        print(ad)

        reference_samples = list(ad.obs[ad.obs['dataset'] == 'reference']['sample'].unique())
        query_samples = list(ad.obs[ad.obs['dataset'] == 'query']['sample'].unique())


        for i in range(25):
            print('RUN: ', i)
            seed = i * 100

            start_time = time.perf_counter()

            scpss = scPSS(ad.copy(), 'sample', reference_samples, query_samples)
            scpss.harmony_integrate(random_state=seed)
            scpss.find_optimal_parameters()
            scpss.set_distance_and_condition()

            end_time = time.perf_counter()

            ad_que = scpss.adata[scpss.query_mask]

            true_labels = ad_que.obs['zone'].isin(['BZ1', 'BZ2']).to_numpy()
            predicted_labels = ad_que.obs['scpss_condition'].isin(['pathological']).to_numpy()
            dist_que_ref = scpss.adata.obs.loc[scpss.query_mask, 'scpss_scores']
            dist_ref_ref = scpss.adata.obs.loc[scpss.reference_mask, 'scpss_scores']

            time_taken = end_time - start_time


            from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

            accuracy = accuracy_score(true_labels, predicted_labels)
            precision = precision_score(true_labels, predicted_labels)
            recall = recall_score(true_labels, predicted_labels)
            f1 = f1_score(true_labels, predicted_labels)

            print("Accuracy:", accuracy)
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1-score:", f1)

            from sklearn.metrics import roc_auc_score, average_precision_score
            from sklearn.metrics import roc_curve, precision_recall_curve

            y_true = true_labels
            y_scores = dist_que_ref

            auc = roc_auc_score(y_true, y_scores)
            print("AUC:", auc)

            aupr = average_precision_score(y_true, y_scores)
            print("AUPR:", aupr)

            outlier_ratio_ref = np.mean(dist_ref_ref > scpss.best_params['threshold'])
            print("Outlier ratio in reference:", outlier_ratio_ref)
            outlier_ratio_que = np.mean(dist_que_ref > scpss.best_params['threshold'])
            print("Outlier ratio in query:", outlier_ratio_que)


            print(PROBLEM_KEY, seed, accuracy, precision, recall, f1, auc, aupr, outlier_ratio_ref, outlier_ratio_que, time_taken, sep=',', file=f, flush=True)


View of AnnData object with n_obs × n_vars = 6135 × 5000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'integrated_snn_res.0.2', 'seurat_clusters', 'integrated_snn_res.0.6', 'original', 'sample', 'integrated_snn_res.1.2', 'final_cluster', 'orig_id', 'sample_id', 'neworig', 'orig_id2', 'percent.mt', 'total', 'obs_names', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'dataset', 'batch_key', 'zone'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    layers: 'counts'
RUN:  0


2025-07-24 05:49:09,875 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
INFO:harmonypy:Computing initial centroids with sklearn.KMeans...


✅ PCA Complete.


2025-07-24 05:49:12,150 - harmonypy - INFO - sklearn.KMeans initialization complete.
INFO:harmonypy:sklearn.KMeans initialization complete.
2025-07-24 05:49:12,276 - harmonypy - INFO - Iteration 1 of 10
INFO:harmonypy:Iteration 1 of 10
2025-07-24 05:49:15,684 - harmonypy - INFO - Iteration 2 of 10
INFO:harmonypy:Iteration 2 of 10
2025-07-24 05:49:17,493 - harmonypy - INFO - Converged after 2 iterations
INFO:harmonypy:Converged after 2 iterations


✅ Harmony Integration Complete.
✅ Found Optimal Parameters.
✅ Stored distances and conditions in Anndata object.
Accuracy: 0.7428470505121865
Precision: 0.5987370838117106
Recall: 0.9729477611940298
F1-score: 0.7412935323383084
AUC: 0.9155425614961011
AUPR: 0.8371368612012549
Outlier ratio in reference: 0.03238498789346247
Outlier ratio in query: 0.6153302719886966
RUN:  1


...

✅ Harmony Integration Complete.
✅ Found Optimal Parameters.
✅ Stored distances and conditions in Anndata object.
Accuracy: 0.8172549019607843
Precision: 0.8084481175390267
Recall: 0.8068181818181818
F1-score: 0.8076323273094211
AUC: 0.9004761132757021
AUPR: 0.9022892014577595
Outlier ratio in reference: 0.03389830508474576
Outlier ratio in query: 0.4745098039215686
RUN:  24


2025-07-24 06:57:54,930 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
INFO:harmonypy:Computing initial centroids with sklearn.KMeans...


✅ PCA Complete.


2025-07-24 06:57:58,038 - harmonypy - INFO - sklearn.KMeans initialization complete.
INFO:harmonypy:sklearn.KMeans initialization complete.
2025-07-24 06:57:58,152 - harmonypy - INFO - Iteration 1 of 10
INFO:harmonypy:Iteration 1 of 10
2025-07-24 06:58:02,659 - harmonypy - INFO - Iteration 2 of 10
INFO:harmonypy:Iteration 2 of 10
2025-07-24 06:58:08,755 - harmonypy - INFO - Converged after 2 iterations
INFO:harmonypy:Converged after 2 iterations


✅ Harmony Integration Complete.
✅ Found Optimal Parameters.
✅ Stored distances and conditions in Anndata object.
Accuracy: 0.8184749455337691
Precision: 0.8105321303627324
Recall: 0.8068181818181818
F1-score: 0.8086708918894094
AUC: 0.9020463485862706
AUPR: 0.9034880118823194
Outlier ratio in reference: 0.033292978208232446
Outlier ratio in query: 0.4732897603485839


In [10]:
import pandas as pd

In [11]:
metrics = pd.read_csv(metrics_csv_path)

In [12]:
metrics

,PROBLEM_KEY,seed,accuracy,precision,recall,f1,auc,aupr,outlier_ratio_ref,outlier_ratio_que,time_taken
0,ref_0_que_1hr,0,0.742847,0.598737,0.972948,0.741294,0.915543,0.837137,0.032385,0.615330,23.909247
1,ref_0_que_1hr,100,0.742141,0.598163,0.972015,0.740583,0.914858,0.837088,0.032688,0.615330,22.929722
2,ref_0_que_1hr,200,0.726245,0.583007,0.972948,0.729116,0.917683,0.844002,0.033596,0.631932,22.050614
3,ref_0_que_1hr,300,0.732250,0.588600,0.972948,0.733474,0.919080,0.846998,0.031174,0.625927,22.849272
4,ref_0_que_1hr,400,0.738255,0.594410,0.972015,0.737699,0.915937,0.838947,0.032082,0.619216,22.765767
...,...,...,...,...,...,...,...,...,...,...,...
145,ref_0_que_1-7,2000,0.815076,0.807224,0.802786,0.804999,0.899227,0.900805,0.033293,0.472854,52.133960
146,ref_0_que_1-7,2100,0.815948,0.808943,0.802419,0.805668,0.900972,0.901057,0.034806,0.471634,52.960602
147,ref_0_que_1-7,2200,0.816296,0.808287,0.804435,0.806357,0.900610,0.902384,0.033293,0.473203,55.046056
148,ref_0_que_1-7,2300,0.817255,0.808448,0.806818,0.807632,0.900476,0.902289,0.033898,0.474510,51.955876


In [13]:
from mean_margin import mean_margin_by_group
result = mean_margin_by_group(metrics, "PROBLEM_KEY")

In [14]:
result

,PROBLEM_KEY,seed,accuracy,precision,recall,f1,auc,aupr,outlier_ratio_ref,outlier_ratio_que,time_taken
0,ref_0_que_1,1200.0000 ± 303.7976,0.8436 ± 0.0014,0.7939 ± 0.0021,0.9584 ± 0.0011,0.8684 ± 0.0010,0.9509 ± 0.0002,0.9591 ± 0.0002,0.0277 ± 0.0010,0.6498 ± 0.0023,26.0549 ± 0.6884
1,ref_0_que_1-7,1200.0000 ± 303.7976,0.8164 ± 0.0006,0.8063 ± 0.0014,0.8080 ± 0.0022,0.8072 ± 0.0008,0.9012 ± 0.0004,0.9024 ± 0.0003,0.0346 ± 0.0008,0.4765 ± 0.0020,52.7172 ± 0.6244
2,ref_0_que_1hr,1200.0000 ± 303.7976,0.7374 ± 0.0021,0.5937 ± 0.0020,0.9720 ± 0.0004,0.7371 ± 0.0015,0.9176 ± 0.0015,0.8426 ± 0.0033,0.0323 ± 0.0004,0.6200 ± 0.0022,22.5685 ± 0.3859
3,ref_0_que_3,1200.0000 ± 303.7976,0.7644 ± 0.0036,0.6352 ± 0.0039,0.9788 ± 0.0018,0.7704 ± 0.0025,0.9322 ± 0.0006,0.8944 ± 0.0011,0.0390 ± 0.0014,0.6222 ± 0.0046,16.4271 ± 0.4041
4,ref_0_que_4hr,1200.0000 ± 303.7976,0.7995 ± 0.0020,0.7580 ± 0.0019,0.9923 ± 0.0005,0.8595 ± 0.0012,0.9403 ± 0.0006,0.9591 ± 0.0004,0.0246 ± 0.0003,0.8088 ± 0.0021,16.5552 ± 0.4214
5,ref_0_que_7,1200.0000 ± 303.7976,0.6410 ± 0.0011,0.5582 ± 0.0008,0.9616 ± 0.0012,0.7064 ± 0.0006,0.8511 ± 0.0027,0.8239 ± 0.0040,0.0325 ± 0.0018,0.7737 ± 0.0017,21.0127 ± 0.3902


In [15]:
result.to_csv(results_csv_path, index=False)